In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 
%load_ext memory_profiler
from sklearn.metrics import make_scorer
from scipy.special import expit
import time
import math
import random
from memory_profiler import memory_usage
from sklearn import metrics as mt
from sklearn.neural_network import MLPClassifier




from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

target_classifier = 'PC'
df = pd.read_csv('responses.csv', sep=",")

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [3]:
# remove rows whose target classfier value is NaN
df_cleaned_classifier = df[np.isfinite(df[target_classifier])]
# change NaN number values to the mean
df_imputed = df_cleaned_classifier.fillna(df.mean())
# get categorical features
object_features = list(df_cleaned_classifier.select_dtypes(include=['object']).columns)
# one hot encode categorical features
one_hot_df = pd.concat([pd.get_dummies(df_imputed[col],prefix=col) for col in object_features], axis=1)
# drop object features from imputed dataframe
df_imputed_dropped = df_imputed.drop(object_features, 1)
frames = [df_imputed_dropped, one_hot_df]
# concatenate both frames by columns
df_fixed = pd.concat(frames, axis=1)

In [4]:
# Research on Cost Matrix
# http://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.im.model.doc/c_cost_matrix.html

cost_matrix = np.matrix([[0,1,2,3,4],
[1,0,1,2,3],
[3,1,0,1,2],
[5,3,1,0,1],
[7,5,2,1,0]])

def get_confusion_costTot(confusion_matrix, cost_matrix):
    score = np.sum(confusion_matrix*cost_matrix)
    return score

confusion_scorer = make_scorer(get_confusion_costTot, greater_is_better=False)
confusion_scorer

make_scorer(get_confusion_costTot, greater_is_better=False)

In [8]:
from sklearn.model_selection import StratifiedKFold

# we want to predict the X and y data as follows:
if target_classifier in df_fixed:
    y = df_fixed[target_classifier].values # get the label we want
    del df_fixed[target_classifier] # get rid of the class label
    X = df_fixed.values # use everything else to predict!

X = X/5
num_folds = 10

cv_object = StratifiedKFold(n_splits= num_folds, random_state=None, shuffle=True)
cv_object.split(X,y)

print(cv_object)

StratifiedKFold(n_splits=10, random_state=None, shuffle=True)


In [11]:
class MyEnsemble():
    
    def __init__(self, c, X, y):
        self.c = c
        self._w = []
        self.X = X
        self.y = y
        self._p = []
        self._bags
        
    def predict(self, x):
        self._p = np.hstack(predict_proba(x))
        totals = self._p.T * self.w
        return(np.amax(totals))    
    

    def fit(self):
        i = 0
        for n in self.c:
            for classifier, bags in zip(self.c[i], self._bags[i]):
                # want to fit each classifier with each vector in bag
                classifier.predict_proba(bags)
            i+=1
    

    def _sample(self):
        self._bags = self.X[np.random.randint(len(self.c), 20), :]
    

In [18]:
clfs = []
clf_amt = 20

for i in range(0, clf_amt):

    clf = MLPClassifier(hidden_layer_sizes=(50, ), 
                            activation='relu', # type of non-linearity, every layer
                            solver='sgd', 
                            alpha=1e-4, # L2 penalty
                            batch_size= 'auto', # min of 200, num_samples
                            learning_rate='constant', # adapt learning? only for sgd
                            learning_rate_init=0.1, # only SGD
                            power_t=0.0,    # only SGD with inverse scaling of learning rate
                            max_iter=75, # stopping criteria
                            shuffle=True, 
                            random_state=1, 
                            tol=0, # for stopping
                            verbose=False, 
                            warm_start=False, 
                            momentum=0.9, # only SGD
                            nesterovs_momentum=False, # only SGD
                            early_stopping=False, 
                            validation_fraction=0.0, # only if early_stop is true
                            beta_1=0.9, # adam decay rate of moment
                            beta_2=0.999, # adam decay rate of moment
                            epsilon=1e-08) # adam numerical stabilizer
    
    clfs.append(clf)



In [19]:
print(len(clfs))

20


In [ ]:
ensemble = MyEnsemble(clfs, X, y)